In [1]:
import requests
st = requests.get('https://www.cars.com/shopping/results/?list_price_max=&makes[]=mercedes_benz&maximum_distance=20&models[]=&page=1&stock_type=cpo&zip=')
st.status_code

200

In [2]:
import threading
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
list_title = []
list_mileage = []
list_primary_price = []
list_secondary_price_drop = []
list_dealer_name = []
list_sds_rating_count = []
list_sds_rating_link_sds_button_link = []
list_link = []
class Crawler:
    def __init__(self, urls):
        self.urls = urls
        self.lock = threading.Lock()

    def crawl(self):
        while True:
            with self.lock:
                if not self.urls:
                    break
                url = self.urls.pop()

            response = requests.get(url)
            sub = BeautifulSoup(response.text, 'html.parser')
            tagets = sub.find_all('div', class_='vehicle-card-main js-gallery-click-card')
            for i in tagets:
                title = i.find('h2', class_='title').text
                link = i.find('a', class_='vehicle-card-link js-gallery-click-link').get('href')
                mileage = i.find('div', class_='mileage').text
                primary_price = i.find('span', class_='primary-price').text
                dealer_name = i.find('div', class_='dealer-name').text.strip()
                if i.find('span', class_='sds-rating__count') is None:
                    sds_rating_count = 'sds-rating--not-rated'
                else:
                    sds_rating_count = i.find('span', class_='sds-rating__count').text
                sds_rating_link_sds_button_link = i.find('span', class_='sds-rating__link sds-button-link').text.replace('(', '').replace(')', '')
                list_title.append(title)
                list_mileage.append(mileage)
                list_primary_price.append(primary_price)
                list_dealer_name.append(dealer_name)
                list_sds_rating_count.append(sds_rating_count)
                list_sds_rating_link_sds_button_link.append(sds_rating_link_sds_button_link)
                list_link.append('https://www.cars.com'+link)
                
            

if __name__ == '__main__':
    urls = [f'https://www.cars.com/shopping/results/?list_price_max=&makes[]=mercedes_benz&maximum_distance=20&models[]=&page={i}&stock_type=cpo&zip='.format(i) for i in range(1,101)]
    crawlers = [Crawler(urls) for _ in range(6)]
    threads = [threading.Thread(target=crawler.crawl) for crawler in crawlers]

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

In [3]:
len(list_title), len(list_mileage), len(list_primary_price), len(list_dealer_name), len(list_sds_rating_count), len(list_sds_rating_link_sds_button_link), len(list_link)

(2000, 2000, 2000, 2000, 2000, 2000, 2000)

In [4]:
df = pd.DataFrame({'title':list_title, 'mileage':list_mileage, 'primary_price':list_primary_price, 'dealer_name':list_dealer_name, 'sds_rating_count':list_sds_rating_count, 'sds_rating_link_sds_button_link':list_sds_rating_link_sds_button_link, 'link':list_link})

In [5]:
df.to_csv('cars.csv', index=False)